# Two different ways of importing the models and making predictions

### Loading the model/params/checkpoint by giving paths: preferred

In [1]:
import context
from mlnowcasting.predict import predict

Giving minimum arguments. It uses default paths and name conventions.

Here it's assuming that:
 - the dataset is in the default location "mlnowcasting/data/datasets/"
 - the checkpoint is in the default location "mlnowcasting/data/checkpoint/"
 - the checkpoint has the same name as the model (ending in .pth)
 - the parameters file is in the default location "mlnowcasting/data/checkpoint/"
 - the parameters has the same name as the model (ending in .params)

In [2]:
dataset    = 'example_dataset'
model_name = 'convGRU_16_16'
th_dbz = 20 # reflectivity threshold for computing the metrics
N = 0 # sample from dataset
prediction, target = predict(dataset, model_name)
prediction.shape

=> loaded checkpoint 'convGRU_16_16.pth' (trained for 300 epochs)


(10, 16, 64, 64)

Giving all possible arguments

In [3]:
model_name          = 'convGRU_16_16'
dataset             = '../data/datasets/example_dataset.npy'
checkpoint_filename = '../data/checkpoints/convGRU_16_16.pth'
params_filename     = '../data/checkpoints/convGRU_16_16.param'
th_dbz = 20 # reflectivity threshold for computing the metrics
N = 0 # sample from dataset
prediction, target = predict(dataset, model_name, checkpoint_filename, params_filename, N)
prediction.shape

=> loaded checkpoint 'convGRU_16_16.pth' (trained for 300 epochs)


(1, 16, 64, 64)

### Loading the model/params as modules: alternative for developers
It assumes that the paramerter file use for the training remains the same, and it contains the path to the checkpoint.

In [4]:
import context
import torch
import numpy as np
from mlnowcasting.models.architectures.convGRU_16_16 import get_model
from mlnowcasting.models.params.example_params import params
from mlnowcasting.utils.torch_trainer import Torch_Trainer
from mlnowcasting.utils.utils import any2reflectivity

In [5]:
testing_dataset = '../data/datasets/example_dataset.npy'
th_dbz = 20 # reflectivity threshold for computing the metrics
N = 0 # sample from testing_dataset

In [6]:
with torch.no_grad():
    T = Torch_Trainer(None, None, get_model, None)
    T.load_checkpoint(params)
    device = torch.device("cuda:0")
    test = np.load(testing_dataset)
    fi = params['in_frames']
    fo = params['out_frames']
    context = np.expand_dims(test[:,:fi],1)/255
    target  = np.expand_dims(test[:,fi:fi+fo],1)/255
    target  = torch.tensor(target).to(device).float()
    prediction = T.model(torch.tensor(context).to(device).float())
    target  = any2reflectivity(target)
    prediction = any2reflectivity(prediction)
prediction.shape

=> loaded checkpoint 'convGRU_16_16.pth' (trained for 300 epochs)


(10, 1, 16, 64, 64)